In [1]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [2]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [3]:
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [4]:
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [5]:
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [6]:
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [7]:
def emojize(df):
    for i in range(len(df)):
        if df['Hot'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n🔥')
        if df['Hot'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Hot'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('🔥')
        if df['Cold'][i+1] > 0.0: df['Name'][i+1] = df['Name'][i+1] + ('\n❄️')
        if df['Cold'][i+1] >= 1.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
        if df['Cold'][i+1] >= 2.0: df['Name'][i+1] = df['Name'][i+1] + ('❄️')
    return df

In [8]:
def gen_page(df, out, coloring):
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    images = []
    for i in df["Name"]:
        code = "https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg"
        if img_dict.get(i) != None:
            code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg"
        images.append(code)
    df["Picture"] = images
    
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    pre = (
        "<head>\n<script src=\"sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<input type=\"text\" id=\"position\" onkeyup=\"posSearch()\" placeholder=\"Search for position...\">\n"
        "</div>\n\n"
    )
    
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    # render dataframe as html
    if coloring is not None:
        images = ['<img src="' + image + '" width="60">' for image in images]
        df["Picture"] = images
        df = emojize(df)
        body = df.style.apply(coloring, axis=1).hide_columns(['Hot', 'Cold']).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data

    body = body.replace("<table id=", "<table ignore=", 1)
    body = body.replace("<table ", "<table id=\"tbl\" class=\"sortable\" ", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)
    return df

In [9]:
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', highlight_fd)
gen_page(st, 'st.html', None)

C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\kunal_tg\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

,Name,Picture,Position,Game,Team,Minutes,2PT FG,3PT FG,FTM,Rebounds,Assists,Blocks,Steals,Turnovers,Opponent Defensive Rank vs Position,Injury Indicator,Injury Details
1,LeBron James,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/jamesle01.jpg,SF,LAC@LAL,LAL,43.74,9.23,2.72,7.18,10.79,12.61,0.96,2.97,3.60,7.0,,
2,Anthony Davis,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/davisan02.jpg,PF,LAC@LAL,LAL,36.53,11.73,0.70,5.71,9.30,2.65,1.65,2.01,1.86,14.0,GTD,Eye
3,Kawhi Leonard,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/leonaka01.jpg,SF,LAC@LAL,LAC,35.54,8.54,2.14,7.84,7.23,3.49,0.78,1.95,2.48,7.0,,
4,Jrue Holiday,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/holidjr01.jpg,SG,UTA@NO,NO,27.51,3.94,2.69,3.82,4.77,7.64,0.00,1.95,2.98,15.0,,
5,Rudy Gobert,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/goberru01.jpg,C,UTA@NO,UTA,30.23,4.94,0.11,2.92,10.87,1.19,1.96,0.90,2.52,14.0,,
6,Lonzo Ball,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/balllo01.jpg,PG,UTA@NO,NO,31.57,1.72,2.42,1.69,8.00,7.34,0.26,2.19,4.60,15.0,,
7,Paul George,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/georgpa01.jpg,SG,LAC@LAL,LAC,33.85,6.54,2.04,5.80,5.34,3.24,0.40,1.93,2.54,7.0,,
8,Brandon Ingram,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/ingrabr01.jpg,SF,UTA@NO,NO,24.42,4.64,1.76,1.61,5.88,3.70,0.31,1.31,1.95,13.0,GTD,Face
9,Donovan Mitchell,https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/mitchdo01.jpg,SG,UTA@NO,UTA,34.26,7.04,1.59,3.93,4.30,2.44,0.38,2.38,1.45,19.0,,
10,Zion Williamson,https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg,PF,UTA@NO,NO,17.12,4.20,1.52,2.42,5.35,1.57,0.57,0.00,3.36,13.0,O,Personal
